In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import csv
import math
import matplotlib as plt

In [2]:
batch_size=32
csv_file="atlas-higgs-challenge-2014-v2.csv"
crossval_nbr=5


In [3]:
class Dataset_Cust(torch.utils.data.Dataset):
    def __init__(self, csv_file, train=True, crossval_nbr=5, transformation=None):
        'Initialization'
        self.all=np.array(pd.read_csv(csv_file))
        #print(self.all[:,:-3])
        self.data=torch.from_numpy(self.all[:,0:-3].astype('float64'))
        self.train=train
        
        self.crossval_nbr=crossval_nbr
        self.testpart_num=crossval_nbr-1
        
        self.nbrpoint=len(self.all)
        #self.nbrtrain=int((1-1/crossval_nbr)*self.nbrpoint)
        
        self.Taille_section=int(self.nbrpoint/self.crossval_nbr)
        self.debut_test=self.testpart_num * self.Taille_section
        
        if self.testpart_num==self.crossval_nbr-1:
            self.fin_test=self.nbrpoint
        else:
            self.fin_test=self.debut_test + self.Taille_section
        
        self.nbrtest=self.fin_test - self.debut_test
        self.nbrtrain=self.nbrpoint - self.nbrtest
        #self.numtest=self.numpoint - self.numtrain
        
        self.transformation=transformation
        

    def __len__(self):
        'Denotes the total number of samples'
        if self.train:
            return self.nbrtrain
        else:
            return self.fin_test - self.debut_test
    

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        if self.train:
            if index>= self.debut_test:
                index+=self.nbrtest
                
        else:
            index+=self.debut_test
        
        #sample=torch.from_numpy(self.all[index][0:-3].astype('float64'))
        sample=self.data[index]
        sample.requires_grad=True
        label=1 if self.all[index][-3]=="s" else 0
        
        if self.transformation :
            sample=self.transformation(sample)

        return sample.float(), float(label)
    
    def Set_Train(self, train):
        self.train=train
        
    def Set_TestPart_Num(self,num):
        self.testpart_num=num
        
        self.debut_test=self.testpart_num * self.Taille_section
        
        if self.testpart_num==self.crossval_nbr-1:
            self.fin_test=self.nbrpoint
        else:
            self.fin_test=self.debut_test + self.Taille_section
        
        self.nbrtest=self.fin_test - self.debut_test
        self.nbrtrain=self.nbrpoint - self.nbrtest

In [4]:
Dataset=Dataset_Cust(csv_file)

In [5]:
Data_Loader=torch.utils.data.DataLoader(Dataset, batch_size=batch_size, shuffle=True)

In [6]:
def AMS(s,b):
    assert s >= 0
    assert b >= 0
    bReg = 10.
    return math.sqrt(2 * ((s + b + bReg) * 
                          math.log(1 + s / (b + bReg)) - s))


In [7]:
class Perceptron(nn.Module):
    def __init__(self):
        super(Perceptron,self).__init__()
        self.fc1=nn.Linear(30,1)
        
    
    def forward(self, x):
        return F.sigmoid(self.fc1(x))

    
class MLP_1(nn.Module):
    def __init__(self):
        super(MLP_1,self).__init__()
        self.fc1=nn.Linear(30,15)
        self.fc2=nn.Linear(15,1)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        return F.sigmoid(self.fc2(x))
        
class MLP_2(nn.Module):
    def __init__(self):
        super(MLP_2,self).__init__()
        self.fc1=nn.Linear(30,15)
        self.fc2=nn.Linear(15,7)
        self.fc3=nn.Linear(7,1)
        
    def forward(self,x):
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        return F.sigmoid(self.fc3(x))

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Conv1d(1,10,5)
        self.conv2=nn.Conv1d(10,20,5)
        self.fc=nn.Linear(50*4*4,1)
        
    def forward(self, x):
        x=F.max_pool1d(F.relu(self.conv1(x)),2,2)
        x=F.max_pool1d(F.relu(self.conv2(x)),2,2)
        x=x.view(-1,50*4*4)
        x=self.fc(x)
        return F.sigmoid(x)

In [8]:
def init_weights(m):
    if type(m) == nn.Linear or type(m)==nn.Conv1d:
        torch.nn.init.zeros_(m.weight)
        torch.nn.init.zeros_(m.bias)


In [9]:
Percep=Perceptron()
Mlp1=MLP_1()
Mlp2=MLP_2()
Cnn=CNN()
#print(Mlp1.fc1.weight)

In [10]:
criterion= nn.BCELoss()
optimizerPercep= optim.SGD(Percep.parameters(), lr=0.001, momentum=0.9)
optimizeMlp1= optim.SGD(Mlp1.parameters(), lr=0.001, momentum=0.9)
optimizeMlp2= optim.SGD(Mlp2.parameters(), lr=0.001, momentum=0.9)
optimizeCnn= optim.SGD(Cnn.parameters(), lr=0.001, momentum=0.9)

In [11]:
def Train(Model, Optimizer, Criterion, Data_Loader,Epoch=10, log_interval=100):
    #Data_Loader.dataset.Set_TestPart_Num(4)
    print(Data_Loader.dataset.debut_test)
    predicted=np.empty(batch_size)
    #print(type(Percep.fc1.weight[0][0].item()))
    #print(type(Percep.fc1.bias[0].item()))
    Model.apply(init_weights)
    
    Data_Loader.dataset.train=True
    
    for epoch in range(Epoch):
        ams=0
        TP=0 #True Positive
        FP=0 # False Positive
        S=0 #True Positive pondéré
        B=0 # False Positive pondéré
        Loss=0
        correct=0
    
        fp=0
        tp=0
        total=0
        running_loss=0
        for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            Optimizer.zero_grad()
            
            #print(inputs[0][0].requires_grad)
            #print(labels.size(0))
            #print(inputs[0][0])
            
            
            outputs= Model(inputs[:,1:31])
        
             #print(outputs[0]+outputs[1])
             # print(outputs.view(-1,32))
            
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= Criterion(outputs, labels.float())
            loss.backward()
            Optimizer.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                if tp+fp>0:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                else:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} ".format(epoch+1, batch_id+1, running_loss/log_interval)) 

                total=0
                tp=0
                fp=0
                running_loss=0
        if TP+FP>0:
            print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
        else:
            print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}  , AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), AMS(S,B) ))
        
    print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f},  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))
    return [Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ]
        

In [12]:
#Res=Train(Mlp2, optimizeMlp2, criterion, Data_Loader )
#print(Res)
a=np.arange(4).reshape(2,2)
a[0]=[2,2]
print(a)
print(np.mean(a, axis=0))

[[2 2]
 [2 3]]
[2.  2.5]


In [13]:
def Test(Model, Criterion, Data_Loader):

    correct= 0
    total= 0
    test_loss = 0
    predicted=np.empty(batch_size)
    TP=0
    FP=0
    S=0
    B=0

    Data_Loader.dataset.train=False
    with torch.no_grad():
        for sample in Data_Loader:
            inputs, labels= sample
            #images=images.view(-1,784)
            outputs= Model(inputs[:,1:31])

            total+= labels.size(0)

            for i in range(labels.size(0)):
                    if outputs.data[i]>0.5:
                        predicted[i]=1

                        if labels[i]==1:
                            TP+=1
                            #tp+=1
                            S+= inputs[i][31]
                            correct+=1
                        else:
                            FP+=1
                            #fp+=1
                            B+=inputs[i][31]
                    else:
                        predicted[i]=0
                        if labels[i]==0:
                            correct+=1

            test_loss+=Criterion(outputs,labels.float())




    test_loss/= len(Data_Loader.dataset)

    print("Résultat test: {} bonnes classification sur {} données de test:loss moyen={:.8f} Accuracy={:.3f}%, Precision={:.3f}%, AMS={:.3f}".format(correct,total, test_loss/total,100*correct/total, 100*TP/(TP+FP), AMS(S,B)))
    
    return [test_loss/total, 100*correct/total, 100*TP/(TP+FP), AMS(S,B)]


In [14]:
######TEST Perceptron
Tableau_Perceptron=np.arange(crossval_nbr*4).reshape(crossval_nbr,4)
for section in range(crossval_nbr):
    print("Train {} :\n".format(section))
    Data_Loader.dataset.Set_TestPart_Num(section)
    Train(Percep, optimizerPercep, criterion, Data_Loader)
    print("Test {} :\n".format(section))
    Tableau_Perceptron[section]=Test(Percep, criterion, Data_Loader)

Res_Perceptron=mean(Tableau_Perceptron,axis=0)

print("tableau pour les {} étapes de validations:\n".format(crossval_nbr))
print(Tableau_Perceptron)

print("Moyennne des metrics:\n")
print(Res_Perceptron)

Train 0 :

0
epoch:1, nombre batch:1 loss:0.007 


/home/andrei/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch:1, nombre batch:101 loss:9.765 precision=46.895%
epoch:1, nombre batch:201 loss:10.031 precision=45.302%
epoch:1, nombre batch:301 loss:10.244 precision=45.740%
epoch:1, nombre batch:401 loss:9.731 precision=46.369%
epoch:1, nombre batch:501 loss:9.723 precision=47.101%
epoch:1, nombre batch:601 loss:9.783 precision=46.889%
epoch:1, nombre batch:701 loss:10.353 precision=44.246%
epoch:1, nombre batch:801 loss:9.921 precision=48.977%
epoch:1, nombre batch:901 loss:9.911 precision=48.741%
epoch:1, nombre batch:1001 loss:9.619 precision=46.091%
epoch:1, nombre batch:1101 loss:9.930 precision=46.871%
epoch:1, nombre batch:1201 loss:10.096 precision=45.094%
epoch:1, nombre batch:1301 loss:10.103 precision=42.715%
epoch:1, nombre batch:1401 loss:9.671 precision=47.039%
epoch:1, nombre batch:1501 loss:9.982 precision=45.870%
epoch:1, nombre batch:1601 loss:10.184 precision=45.289%
epoch:1, nombre batch:1701 loss:9.671 precision=48.376%
epoch:1, nombre batch:1801 loss:10.174 precision=46

KeyboardInterrupt: 

In [29]:
########Test MLP à une Couche Cachée
Tableau_Mlp1=np.arange(crossval_nbr*4).reshape(crossval_nbr,4)
for section in range(crossval_nbr):
    print("Train {} :\n".format(section))
    Data_Loader.dataset.Set_TestPart_Num(section)
    Train(Mlp1, optimizeMlp1, criterion, Data_Loader)
    print("Test {} :\n".format(section))
    Tableau_Mlp1[section]=Test(Mlp1, criterion, Data_Loader)

Res_Mlp1=mean(Tableau_Mlp1,axis=0)

print("tableau pour les {} étapes de validations:\n".format(crossval_nbr))
print(Tableau_Mlp1)

print("Moyennne des metrics:\n")
print(Res_Mlp1)

Train 0 :

0
epoch:1, nombre batch:1 loss:0.007 
epoch:1, nombre batch:101 loss:0.684 
epoch:1, nombre batch:201 loss:0.668 
epoch:1, nombre batch:301 loss:0.661 
epoch:1, nombre batch:401 loss:0.653 
epoch:1, nombre batch:501 loss:0.650 
epoch:1, nombre batch:601 loss:0.647 
epoch:1, nombre batch:701 loss:0.645 
epoch:1, nombre batch:801 loss:0.649 
epoch:1, nombre batch:901 loss:0.656 
epoch:1, nombre batch:1001 loss:0.638 
epoch:1, nombre batch:1101 loss:0.640 
epoch:1, nombre batch:1201 loss:0.645 
epoch:1, nombre batch:1301 loss:0.644 
epoch:1, nombre batch:1401 loss:0.644 
epoch:1, nombre batch:1501 loss:0.633 
epoch:1, nombre batch:1601 loss:0.632 
epoch:1, nombre batch:1701 loss:0.637 
epoch:1, nombre batch:1801 loss:0.655 
epoch:1, nombre batch:1901 loss:0.642 
epoch:1, nombre batch:2001 loss:0.637 
epoch:1, nombre batch:2101 loss:0.647 
epoch:1, nombre batch:2201 loss:0.632 
epoch:1, nombre batch:2301 loss:0.644 
epoch:1, nombre batch:2401 loss:0.630 
epoch:1, nombre batch:25

KeyboardInterrupt: 

In [ ]:
########Test MLP à 2 couches cachées
Tableau_Mlp2=np.arange(20).reshape(crossval_nbr,4)
for section in range(crossval_nbr):
    print("Train {} :\n".format(section))
    Data_Loader.dataset.Set_TestPart_Num(section)
    Train(Mlp2, optimizeMlp2, criterion, Data_Loader)
    print("Test {} :\n".format(section))
    Tableau_Mlp2[section]=Test(Mlp2, criterion, Data_Loader)

Res_Mlp2=mean(Tableau_Mlp2,axis=0)

print("tableau pour les {} étapes de validations:\n".format(crossval_nbr))
print(Tableau_Mlp2)

print("Moyennne des metrics:\n")
print(Res_Mlp2)

In [32]:
Train_Loader.dataset.rate_train=0.9
print(Train_Loader.dataset.rate_train)
print(type(float(int(1))))
#print(torch.tensor(1 if [1,0,0.5,0.4,0.7][i] > 0.5 else 0 for i in range(5)))
#print(torch.tensor([1 if i==2 else 0 for i in range(7)]))

NameError: name 'Train_Loader' is not defined

In [30]:
#TRAIN Perceptron

Epoch=10
#Data_Loader.dataset.Set_TestPart_Num(4)
print(Data_Loader.dataset.debut_test)
log_interval=100
predicted=np.empty(batch_size)
#print(type(Percep.fc1.weight[0][0].item()))
#print(type(Percep.fc1.bias[0].item()))
for epoch in range(Epoch):
    ams=0
    TP=0 #True Positive
    FP=0 # False Positive
    S=0 #True Positive pondéré
    B=0 # False Positive pondéré
    Loss=0
    correct=0
    
    fp=0
    tp=0
    total=0
    running_loss=0
    for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            optimizerPercep.zero_grad()
            
            #print(inputs[0][0].requires_grad)
            #print(labels.size(0))
            #print(inputs[0][0])
            
            
            outputs= Percep(inputs[:,1:31])
            
            #print(outputs[0]+outputs[1])
           # print(outputs.view(-1,32))
        
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= criterion(outputs, labels.float())
            loss.backward()
            optimizerPercep.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                if tp+fp>0:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                else:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} ".format(epoch+1, batch_id+1, running_loss/log_interval)) 

                total=0
                tp=0
                fp=0
                running_loss=0
    if TP+FP>0:
        print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
    else:
        print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}  , AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), AMS(S,B) ))
        
print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f},  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))

        

0
epoch:1, nombre batch:1 loss:0.174 precision=30.000%
epoch:1, nombre batch:101 loss:10.202 precision=44.124%
epoch:1, nombre batch:201 loss:10.029 precision=46.552%
epoch:1, nombre batch:301 loss:9.835 precision=49.428%
epoch:1, nombre batch:401 loss:10.137 precision=45.815%
epoch:1, nombre batch:501 loss:9.999 precision=44.369%
epoch:1, nombre batch:601 loss:9.844 precision=48.600%
epoch:1, nombre batch:701 loss:9.582 precision=48.686%
epoch:1, nombre batch:801 loss:9.783 precision=48.866%
epoch:1, nombre batch:901 loss:10.405 precision=44.659%
epoch:1, nombre batch:1001 loss:10.399 precision=44.766%
epoch:1, nombre batch:1101 loss:9.908 precision=46.658%
epoch:1, nombre batch:1201 loss:9.904 precision=45.125%
epoch:1, nombre batch:1301 loss:9.973 precision=47.026%
epoch:1, nombre batch:1401 loss:9.921 precision=47.362%
epoch:1, nombre batch:1501 loss:10.327 precision=46.028%
epoch:1, nombre batch:1601 loss:9.519 precision=49.718%
epoch:1, nombre batch:1701 loss:10.344 precision=45.

KeyboardInterrupt: 

In [99]:
torch.save(Percep.state_dict(), 'Percep_weights.pth')
Percep.load_state_dict(torch.load('Percep_weights.pth'))
print(Percep.fc1.state_dict())

<All keys matched successfully>

In [16]:
#TRAIN MLP 1 couche

Epoch=10

log_interval=100
predicted=np.empty(batch_size)
#print(type(Percep.fc1.weight[0][0].item()))
#print(type(Percep.fc1.bias[0].item()))
for epoch in range(Epoch):
    ams=0
    TP=0 #True Positive
    FP=0 # False Positive
    S=0 #True Positive pondéré
    B=0 # False Positive pondéré
    Loss=0
    correct=0
    
    fp=0
    tp=0
    total=0
    running_loss=0
    for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            optimizeMlp1.zero_grad()
            
            #print(inputs[0].requires_grad)
            #print(labels.size(0))
            
            
            #print(inputs[0])
            outputs= Mlp1(inputs[:,1:31])
            #print(outputs)
            
            #print(outputs[0]+outputs[1])
           # print(outputs.view(-1,32))
        
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= criterion(outputs, labels.float())
            loss.backward()
            optimizeMlp1.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                if tp+fp>0:
                    print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                total=0
                tp=0
                fp=0
                running_loss=0
    print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}%,  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
   
        
print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f}%,  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Train_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))

        

epoch:1, nombre batch:1 loss:0.046 precision=50.000%
epoch:1, nombre batch:101 loss:6.025 precision=63.537%
epoch:1, nombre batch:201 loss:5.808 precision=59.711%
epoch:1, nombre batch:301 loss:6.074 precision=66.087%
epoch:1, nombre batch:401 loss:5.911 precision=60.561%
epoch:1, nombre batch:501 loss:5.765 precision=63.265%
epoch:1, nombre batch:601 loss:5.866 precision=61.475%
epoch:1, nombre batch:701 loss:5.893 precision=63.261%
epoch:1, nombre batch:801 loss:5.883 precision=54.140%
epoch:1, nombre batch:901 loss:6.087 precision=62.635%
epoch:1, nombre batch:1001 loss:5.889 precision=67.364%
epoch:1, nombre batch:1101 loss:5.639 precision=60.893%
epoch:1, nombre batch:1201 loss:6.032 precision=63.992%
epoch:1, nombre batch:1301 loss:6.424 precision=65.485%
epoch:1, nombre batch:1401 loss:5.948 precision=63.692%
epoch:1, nombre batch:1501 loss:6.135 precision=65.217%
epoch:1, nombre batch:1601 loss:6.028 precision=66.017%
epoch:1, nombre batch:1701 loss:5.905 precision=61.553%
epoc

KeyboardInterrupt: 

In [13]:
print(TP)
print(FP)
print(TP/(TP+FP))

0
0


ZeroDivisionError: division by zero

In [12]:
#TRAIN MLP 2 couche

Epoch=10

log_interval=100
predicted=np.empty(batch_size)
#print(type(Percep.fc1.weight[0][0].item()))
#print(type(Percep.fc1.bias[0].item()))
for epoch in range(Epoch):
    ams=0
    TP=0 #True Positive
    FP=0 # False Positive
    S=0 #True Positive pondéré
    B=0 # False Positive pondéré
    Loss=0
    correct=0
    
    fp=0
    tp=0
    total=0
    running_loss=0
    for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            optimizeMlp2.zero_grad()
            
            #print(inputs)
            #print(labels.size(0))
            
            
            
            outputs= Mlp2(inputs[:,1:31])
            
            #print(outputs[0]+outputs[1])
           # print(outputs.view(-1,32))
        
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= criterion(outputs, labels.float())
            loss.backward()
            optimizeMlp2.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                total=0
                tp=0
                fp=0
                running_loss=0
    print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}%,  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
   
        
print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f}%,  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))

        

/home/andrei/.local/lib/python3.6/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


epoch:1, nombre batch:1 loss:0.046 precision=50.000%
epoch:1, nombre batch:101 loss:14.304 precision=32.933%
epoch:1, nombre batch:201 loss:13.472 precision=37.787%
epoch:1, nombre batch:301 loss:13.849 precision=36.504%
epoch:1, nombre batch:401 loss:13.346 precision=36.445%
epoch:1, nombre batch:501 loss:13.675 precision=37.398%
epoch:1, nombre batch:601 loss:13.911 precision=35.283%
epoch:1, nombre batch:701 loss:13.431 precision=37.949%
epoch:1, nombre batch:801 loss:13.442 precision=37.444%
epoch:1, nombre batch:901 loss:13.772 precision=36.706%
epoch:1, nombre batch:1001 loss:13.895 precision=35.990%
epoch:1, nombre batch:1101 loss:14.226 precision=35.248%
epoch:1, nombre batch:1201 loss:13.702 precision=36.889%
epoch:1, nombre batch:1301 loss:13.944 precision=36.125%
epoch:1, nombre batch:1401 loss:14.245 precision=34.044%
epoch:1, nombre batch:1501 loss:13.963 precision=36.273%
epoch:1, nombre batch:1601 loss:13.787 precision=35.985%
epoch:1, nombre batch:1701 loss:14.125 preci

epoch:1, nombre batch:14401 loss:13.574 precision=36.911%
epoch:1, nombre batch:14501 loss:13.547 precision=37.696%
epoch:1, nombre batch:14601 loss:14.315 precision=35.030%
epoch:1, nombre batch:14701 loss:14.329 precision=33.665%
epoch:1, nombre batch:14801 loss:13.539 precision=37.766%
epoch:1, nombre batch:14901 loss:14.022 precision=36.508%
epoch:1, nombre batch:15001 loss:13.682 precision=36.439%
epoch:1, nombre batch:15101 loss:13.966 precision=35.960%
epoch:1, nombre batch:15201 loss:13.655 precision=36.761%
epoch:1, nombre batch:15301 loss:13.520 precision=37.524%
epoch:1, nombre batch:15401 loss:14.462 precision=34.254%
epoch:1, nombre batch:15501 loss:14.195 precision=35.111%
epoch:1, nombre batch:15601 loss:14.295 precision=35.424%
epoch:1, nombre batch:15701 loss:13.480 precision=37.633%
epoch:1, nombre batch:15801 loss:13.715 precision=36.513%
epoch:1, nombre batch:15901 loss:14.344 precision=35.733%
epoch:1, nombre batch:16001 loss:13.909 precision=35.586%
epoch:1, nombr

/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([30])) that is different to the input size (torch.Size([30, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Resultat Epoch 1;  loss:13.904, Accuracy=42.717%,  precision:36.189%, AMS=0.9860
epoch:2, nombre batch:1 loss:0.148 precision=34.483%
epoch:2, nombre batch:101 loss:14.119 precision=36.106%
epoch:2, nombre batch:201 loss:13.634 precision=36.305%
epoch:2, nombre batch:301 loss:13.780 precision=37.638%
epoch:2, nombre batch:401 loss:13.619 precision=37.893%
epoch:2, nombre batch:501 loss:13.490 precision=38.410%
epoch:2, nombre batch:601 loss:13.916 precision=35.845%
epoch:2, nombre batch:701 loss:14.454 precision=34.755%
epoch:2, nombre batch:801 loss:14.049 precision=36.201%
epoch:2, nombre batch:901 loss:13.595 precision=37.528%
epoch:2, nombre batch:1001 loss:14.076 precision=36.784%
epoch:2, nombre batch:1101 loss:13.705 precision=37.425%
epoch:2, nombre batch:1201 loss:13.858 precision=37.080%
epoch:2, nombre batch:1301 loss:13.799 precision=35.413%
epoch:2, nombre batch:1401 loss:14.253 precision=34.895%
epoch:2, nombre batch:1501 loss:14.106 precision=36.016%
epoch:2, nombre batc

...

epoch:7, nombre batch:12701 loss:14.034 precision=36.138%
epoch:7, nombre batch:12801 loss:13.631 precision=37.387%
epoch:7, nombre batch:12901 loss:14.238 precision=35.006%
epoch:7, nombre batch:13001 loss:13.760 precision=37.243%
epoch:7, nombre batch:13101 loss:13.852 precision=36.836%
epoch:7, nombre batch:13201 loss:13.769 precision=38.004%
epoch:7, nombre batch:13301 loss:14.201 precision=35.334%
epoch:7, nombre batch:13401 loss:14.063 precision=35.920%
epoch:7, nombre batch:13501 loss:14.098 precision=36.394%
epoch:7, nombre batch:13601 loss:13.930 precision=36.844%
epoch:7, nombre batch:13701 loss:13.868 precision=36.826%
epoch:7, nombre batch:13801 loss:13.509 precision=38.178%
epoch:7, nombre batch:13901 loss:13.580 precision=37.371%
epoch:7, nombre batch:14001 loss:14.001 precision=36.600%
epoch:7, nombre batch:14101 loss:13.720 precision=37.401%
epoch:7, nombre batch:14201 loss:13.982 precision=37.121%
epoch:7, nombre batch:14301 loss:13.769 precision=38.169%
epoch:7, nombr

epoch:8, nombre batch:3801 loss:13.880 precision=36.494%
epoch:8, nombre batch:3901 loss:13.200 precision=39.925%
epoch:8, nombre batch:4001 loss:13.874 precision=37.657%
epoch:8, nombre batch:4101 loss:13.870 precision=36.789%
epoch:8, nombre batch:4201 loss:13.722 precision=37.392%
epoch:8, nombre batch:4301 loss:13.772 precision=36.890%
epoch:8, nombre batch:4401 loss:14.272 precision=35.786%
epoch:8, nombre batch:4501 loss:14.272 precision=34.951%
epoch:8, nombre batch:4601 loss:13.648 precision=37.739%
epoch:8, nombre batch:4701 loss:13.942 precision=37.230%
epoch:8, nombre batch:4801 loss:13.841 precision=37.048%
epoch:8, nombre batch:4901 loss:13.608 precision=37.216%
epoch:8, nombre batch:5001 loss:13.609 precision=37.112%
epoch:8, nombre batch:5101 loss:14.216 precision=35.083%
epoch:8, nombre batch:5201 loss:13.646 precision=36.141%
epoch:8, nombre batch:5301 loss:13.751 precision=37.299%
epoch:8, nombre batch:5401 loss:13.754 precision=37.706%
epoch:8, nombre batch:5501 loss

epoch:9, nombre batch:9101 loss:13.784 precision=37.158%
epoch:9, nombre batch:9201 loss:14.286 precision=36.829%
epoch:9, nombre batch:9301 loss:13.737 precision=38.058%
epoch:9, nombre batch:9401 loss:14.527 precision=34.471%
epoch:9, nombre batch:9501 loss:13.522 precision=38.088%
epoch:9, nombre batch:9601 loss:13.691 precision=36.888%
epoch:9, nombre batch:9701 loss:13.712 precision=36.953%
epoch:9, nombre batch:9801 loss:14.032 precision=35.501%
epoch:9, nombre batch:9901 loss:14.040 precision=35.564%
epoch:9, nombre batch:10001 loss:14.598 precision=34.538%
epoch:9, nombre batch:10101 loss:13.998 precision=35.814%
epoch:9, nombre batch:10201 loss:13.869 precision=35.965%
epoch:9, nombre batch:10301 loss:13.591 precision=37.353%
epoch:9, nombre batch:10401 loss:14.131 precision=36.539%
epoch:9, nombre batch:10501 loss:14.027 precision=35.447%
epoch:9, nombre batch:10601 loss:13.747 precision=36.796%
epoch:9, nombre batch:10701 loss:14.152 precision=35.356%
epoch:9, nombre batch:1

epoch:10, nombre batch:101 loss:13.699 precision=37.837%
epoch:10, nombre batch:201 loss:13.740 precision=35.778%
epoch:10, nombre batch:301 loss:13.846 precision=36.259%
epoch:10, nombre batch:401 loss:13.702 precision=37.826%
epoch:10, nombre batch:501 loss:13.519 precision=37.895%
epoch:10, nombre batch:601 loss:13.931 precision=36.500%
epoch:10, nombre batch:701 loss:14.056 precision=35.440%
epoch:10, nombre batch:801 loss:13.482 precision=38.616%
epoch:10, nombre batch:901 loss:13.903 precision=37.328%
epoch:10, nombre batch:1001 loss:13.531 precision=37.732%
epoch:10, nombre batch:1101 loss:13.746 precision=37.686%
epoch:10, nombre batch:1201 loss:14.314 precision=35.443%
epoch:10, nombre batch:1301 loss:13.502 precision=37.403%
epoch:10, nombre batch:1401 loss:14.154 precision=35.036%
epoch:10, nombre batch:1501 loss:13.847 precision=37.402%
epoch:10, nombre batch:1601 loss:13.629 precision=37.148%
epoch:10, nombre batch:1701 loss:13.621 precision=37.684%
epoch:10, nombre batch:

epoch:10, nombre batch:14201 loss:14.060 precision=36.468%
epoch:10, nombre batch:14301 loss:14.211 precision=34.939%
epoch:10, nombre batch:14401 loss:14.036 precision=35.274%
epoch:10, nombre batch:14501 loss:13.931 precision=37.285%
epoch:10, nombre batch:14601 loss:13.653 precision=37.217%
epoch:10, nombre batch:14701 loss:13.572 precision=37.861%
epoch:10, nombre batch:14801 loss:14.092 precision=36.242%
epoch:10, nombre batch:14901 loss:14.356 precision=35.822%
epoch:10, nombre batch:15001 loss:14.027 precision=36.513%
epoch:10, nombre batch:15101 loss:13.771 precision=36.838%
epoch:10, nombre batch:15201 loss:14.316 precision=35.627%
epoch:10, nombre batch:15301 loss:14.236 precision=35.104%
epoch:10, nombre batch:15401 loss:13.852 precision=37.068%
epoch:10, nombre batch:15501 loss:14.234 precision=35.821%
epoch:10, nombre batch:15601 loss:14.237 precision=35.610%
epoch:10, nombre batch:15701 loss:13.671 precision=36.977%
epoch:10, nombre batch:15801 loss:13.884 precision=36.80

In [ ]:
#TRAIN CNN

Epoch=10

log_interval=100
predicted=np.empty(batch_size)
#print(type(Percep.fc1.weight[0][0].item()))
#print(type(Percep.fc1.bias[0].item()))
for epoch in range(Epoch):
    ams=0
    TP=0 #True Positive
    FP=0 # False Positive
    S=0 #True Positive pondéré
    B=0 # False Positive pondéré
    Loss=0
    correct=0
    
    fp=0
    tp=0
    total=0
    running_loss=0
    for batch_id, sample in enumerate(Data_Loader,0):
            inputs, labels= sample
            optimizeCnn.zero_grad()
            
            #print(inputs)
            #print(labels.size(0))
            
            
            
            outputs= Cnn(inputs[:,1:31])
            
            #print(outputs[0]+outputs[1])
           # print(outputs.view(-1,32))
        
            #predicted= torch.tensor(1 if outputs.data[j] > 0.5 else 0 for j in range(len(outputs.data)))
            total+= labels.size(0)
            #correct+= (predicted== labels).sum().item()
            for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
            
        
            #print(outputs)
            #outputs=outputs.view(-1,10)
            loss= criterion(outputs, labels.float())
            loss.backward()
            optimizeCnn.step()
        
            running_loss+= loss.item()
            Loss+=loss.item()
            if batch_id%log_interval==0:
                print("epoch:{}, nombre batch:{} loss:{:.3f} precision={:.3f}%".format(epoch+1, batch_id+1, running_loss/log_interval, 100*tp/(tp+fp))) 
                total=0
                tp=0
                fp=0
                running_loss=0
    print("Resultat Epoch {};  loss:{:.3f}, Accuracy={:.3f}%,  precision:{:.3f}%, AMS={:.4f}".format(epoch+1, Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset), 100*TP/(TP+FP), AMS(S,B) ))
   
        
print("Résultat Entrainement: {} bonnes classification sur {} données :loss moyen/batch={:.4f}, Accuracy={:.3f}%,  precision={:.3f}%, AMS={:.4f}".format(correct,len(Data_Loader.dataset), Loss/(batch_id+1), 100*correct/len(Data_Loader.dataset) ,100*TP/(TP+FP), AMS(S,B)))

        

In [39]:
#TEST Perceptron

correct= 0
total= 0
test_loss = 0
predicted=np.empty(batch_size)
TP=0
FP=0
S=0
B=0

Data_Loader.dataset.train=False
with torch.no_grad():
    for sample in Data_Loader:
        inputs, labels= sample
        #images=images.view(-1,784)
        outputs= Percep(inputs[:,1:31])
        
        total+= labels.size(0)

        for i in range(labels.size(0)):
                if outputs.data[i]>0.5:
                    predicted[i]=1
            
                    if labels[i]==1:
                        TP+=1
                        #tp+=1
                        S+= inputs[i][31]
                        correct+=1
                    else:
                        FP+=1
                        #fp+=1
                        B+=inputs[i][31]
                else:
                    predicted[i]=0
                    if labels[i]==0:
                        correct+=1
        
        test_loss+=criterion(outputs,labels.float())
       
        


test_loss/= len(Data_Loader.dataset)

print("Résultat test: {} bonnes classification sur {} données de test:loss moyen={:.8f} Accuracy={:.3f}%, Precision={:.3f}%, AMS={:.3f}".format(correct,total, test_loss/total,100*correct/total, 100*TP/(TP+FP), AMS(S,B)))

Résultat test: 115013 bonnes classification sur 163650 données de test:loss moyen=0.00000158 Accuracy=70.280%, Precision=66.881%, AMS=0.317


/home/andrei/.local/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
